In [1]:
import utils

In [2]:
# read the configuration file
cfg = utils.read_config('config.yaml')

In [3]:
# load dataset
allData = utils.load_data(cfg['dataset'])


In [4]:
# process dataset (separate features & classes, split in train and test)

X_cv, X_test, y_cv, y_test = utils.dataPrep(allData, cfg['features'], cfg['target'],
                                                  randomState= 55, testSize= 0.25)

In [5]:
score_matrix, best_parameters = utils.run_CV(cfg, X_cv, y_cv)

In [6]:
score_matrix

,algorithm,params,cv_scores,avg_score
0,RandomForestClassifier,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_...","[0.8939851061070422, 0.8920154946161329]",0.893000
1,LGBMClassifier,"{'learning_rate': 0.01, 'max_depth': 3}","[0.8829989829559804, 0.8815822255592908]",0.882291


In [7]:
best_parameters

{'RandomForestClassifier': {'best_params': {'max_depth': 10,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'n_estimators': 100},
  'F1 micro score': 0.8930003003615876},
 'LGBMClassifier': {'best_params': {'learning_rate': 0.01, 'max_depth': 3},
  'F1 micro score': 0.8822906042576356}}

In [9]:
best_model = utils.run_bestModel(cfg['algorithm'], best_parameters,X_cv, X_test, y_cv, y_test )

In [10]:
best_model

,algorithm,F1 score
0,RandomForestClassifier,0.892704
0,LGBMClassifier,0.881185
